# Test Tool Installers  <a class="tocSkip">

**In this notebook:**

- Test all tool installers.

_Please use a Python 3 kernel for the notebook_

## Dependencies

In [ ]:
# System libraries
import logging, os, sys
import re, shutil
import subprocess
import time
import glob, os
import json
from collections import Counter
from typing import List

# Enable logging
logging.basicConfig(
    format="[%(levelname)s] %(message)s", level=logging.INFO, stream=sys.stdout
)

# logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

from tqdm.notebook import tqdm

# Third-party libraries
import numpy as np
import pandas as pd

log = logging.getLogger()

## Utilities

In [ ]:
RESOURCES_PATH = os.getenv("RESOURCES_PATH", "/resources")
TOOLS_DIR = os.path.join(RESOURCES_PATH, "tools")
WORKSPACE_HOME = os.getenv("WORKSPACE_HOME", "/workspace")


def simplify(text) -> str:
    return (
        re.compile(r"[^a-zA-Z0-9-]")
        .sub("-", " ".join(str(text).split()).strip())
        .lower()
    )


def call(command, **kwargs):
    log.debug("Executing: " + command)
    return subprocess.call(command, shell=True, **kwargs)


def run_tool_installers(
    select_tools: List[str] = None,
    ignore_tools: List[str] = None,
    logs_root_path: str = "/workspace/installer-logs/",
):
    tool_installer_scripts = []
    for filename in os.listdir(TOOLS_DIR):
        if filename.endswith(".sh"):
            tool_installer_scripts.append(os.path.join(TOOLS_DIR, filename))

    os.makedirs(logs_root_path, exist_ok=True)
    for installer_script in tqdm(tool_installer_scripts):
        filename = os.path.splitext(os.path.basename(installer_script))[0]

        if select_tools and filename not in select_tools:
            continue

        if ignore_tools and filename in ignore_tools:
            continue

        log.info("Installing " + filename)

        install_log_file = open(os.path.join(logs_root_path, filename + ".txt"), "w")

        exit_code = call(
            f"{installer_script} --install",
            stderr=subprocess.STDOUT,
            stdout=install_log_file,
        )
        install_log_file.close()
        
        if exit_code == 0:
            log.info(filename + ": installed successfully.")
        else:
            log.info(filename + ": failed to install.")
        
        # Cleanup
        call(
            f"{RESOURCES_PATH}/scripts/clean-layer.sh",
            stderr=subprocess.STDOUT,
            stdout=None,
        )

## Evaluate Tool Installer

In [ ]:
run_tool_installers()

## Check system configuration

In [ ]:
import numpy as np
np.show_config()

In [ ]:
!numba -s